In [16]:
import pandas as pd
import os
from pathlib import Path

In [ ]:
CSV_PATH = 'C:/Users/emili/OneDrive/Pulpit/STUDIA/II STOPIEN/ZPB/EMBED_OpenData_metadata.csv'  #każdy niech swoją ścieżkę da
LABELS_DIR = Path('E:/YOLO_labels/labels/train')  #każdy niech swoją ścieżkę da


In [18]:
LABELS_DIR.mkdir(parents=True, exist_ok=True)

In [19]:
#Parsowanie stringa z kordami ROI na listę bboxów
def parse_roi(roi_coords: str):
    roi_coords = roi_coords.translate({ord(c): None for c in "][)(,"})
    flat = list(map(float, roi_coords.split()))
    return [flat[i:i+4] for i in range(0, len(flat), 4)]


# Wczytanie csv
df = pd.read_csv(CSV_PATH)
df.columns = df.columns.str.strip() #usunięcie spacji z nazw kolumn

C:\Users\emili\AppData\Local\Temp\ipykernel_29520\3415323758.py:9: DtypeWarning: Columns (115,116,117,118,119,120,130,132,133,134,135,136,138,143,147,148,149) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_PATH)


In [15]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  

df.head()


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,AcquisitionContextSequence,AcquisitionTime,0_AnatomicRegionSequence_CodeMeaning,0_AnatomicRegionSequence_CodeValue,0_AnatomicRegionSequence_CodingSchemeDesignator,AnodeTargetMaterial,BitsAllocated,BitsStored,BodyPartExamined,BodyPartThickness,BreastImplantPresent,BurnedInAnnotation,Columns,CompressionForce,ContentTime,DetectorBinning,DetectorConditionsNominalFlag,DetectorTemperature,DetectorType,DistanceSourceToDetector,DistanceSourceToPatient,EntranceDose,EntranceDoseInmGy,EstimatedRadiographicMagnificationFactor,Exposure,ExposureControlMode,ExposureControlModeDescription,ExposureInuAs,ExposureTime,ExposureTimeInuS,FieldOfViewHorizontalFlip,FieldOfViewOrigin,FieldOfViewRotation,FilterMaterial,FilterThicknessMaximum,FilterThicknessMinimum,FilterType,FocalSpots,Grid,HalfValueLayer,HighBit,ImageLaterality,ImageType,ImagerPixelSpacing,ImagesInAcquisition,InstanceNumber,IssuerOfPatientID,KVP,LossyImageCompression,Manufacturer,ManufacturerModelName,Modality,OrganDose,OrganExposed,PaddleDescription,PatientOrientation,PatientSex,PerformedProcedureStepDescription,PerformedProcedureStepStartTime,0_PerformedProtocolCodeSequence_CodeMeaning,0_PerformedProtocolCodeSequence_CodeValue,0_PerformedProtocolCodeSequence_CodingSchemeDesignator,PhotometricInterpretation,PixelIntensityRelationship,PixelIntensityRelationshipSign,PixelPaddingValue,PixelRepresentation,PixelSpacing,PositionerPrimaryAngle,PositionerType,PresentationIntentType,PresentationLUTShape,0_ProcedureCodeSequence_CodeMeaning,0_ProcedureCodeSequence_CodeValue,0_ProcedureCodeSequence_CodingSchemeDesignator,0_ProcedureCodeSequence_CodingSchemeVersion,ProtocolName,QualityControlImage,RelativeXRayExposure,RescaleIntercept,RescaleSlope,RescaleType,Rows,SamplesPerPixel,SeriesDescription,SeriesNumber,SeriesTime,SoftwareVersions,0_SourceImageSequence_0_PurposeOfReferenceCodeSequence_CodeMeaning,0_SourceImageSequence_0_PurposeOfReferenceCodeSequence_CodeValue,0_SourceImageSequence_0_PurposeOfReferenceCodeSequence_CodingSchemeDesignator,0_SourceImageSequence_SpatialLocationsPreserved,SpecificCharacterSet,StudyDescription,StudyID,StudyTime,TimeOfLastDetectorCalibration,0_ViewCodeSequence_CodeMeaning,0_ViewCodeSequence_CodeValue,0_ViewCodeSequence_CodingSchemeDesignator,0_ViewCodeSequence_ViewModifierCodeSequence,ViewPosition,WindowCenter,WindowWidth,XRayTubeCurrent,has_pix_array,category,PositionerPrimaryAngleDirection,PixelPaddingRangeLimit,0_ViewCodeSequence_0_ViewModifierCodeSequence_CodeMeaning,0_ViewCodeSequence_0_ViewModifierCodeSequence_CodeValue,0_ViewCodeSequence_0_ViewModifierCodeSequence_CodingSchemeDesignator,AcquisitionDeviceProcessingCode,AcquisitionDeviceProcessingDescription,DerivationDescription,DetectorConfiguration,FieldOfViewDimensions,FieldOfViewShape,Laterality,Sensitivity,ConversionType,0_RelatedSeriesSequence_0_PurposeOfReferenceCodeSequence_CodeMeaning,0_RelatedSeriesSequence_0_PurposeOfReferenceCodeSequence_CodeValue,0_RelatedSeriesSequence_0_PurposeOfReferenceCodeSequence_CodingSchemeDesignator,CollimatorLeftVerticalEdge,CollimatorLowerHorizontalEdge,CollimatorRightVerticalEdge,CollimatorShape,CollimatorUpperHorizontalEdge,CommentsOnRadiationDose,DetectorActiveDimensions,DetectorActiveShape,DetectorElementPhysicalSize,DetectorElementSpacing,DistanceSourceToEntrance,ExposureStatus,0_IconImageSequence_BitsAllocated,0_IconImageSequence_BitsStored,0_IconImageSequence_Columns,0_IconImageSequence_HighBit,0_IconImageSequence_PhotometricInterpretation,0_IconImageSequence_PixelRepresentation,0_IconImageSequence_Rows,0_IconImageSequence_SamplesPerPixel,VOILUTFunction,WindowCenterWidthExplanation,DetectorDescription,png_path,empi_anon,acc_anon,study_date_anon,cohort_num,LateralityDeriveFlag,ImageLateralityFinal,FinalImageType,spot_mag,png_filename,ROI_coords,SRC_DST,match_level,num_roi,anon_dicom_path
95,105,105,105,[],143714.0,Breast,T-04000,SRT,TUNGSTEN,16.0,12.0,BREAST,66.0,NO,NO,2560.0,25.35474,143714.0,"(""1.0"", ""1.0"")",YES,31.99

In [ ]:
# filtrowanie obrazów z ROI
df = df[
    (df['num_roi'] > 0) &
    (df['FinalImageType'] == '2D') &
    (df['cohort_num'] == 1)  
]

for idx, row in df.iterrows():
    roi_str = row['ROI_coords']
    if not isinstance(roi_str, str) or roi_str.strip() == "":
        continue

    try:
        rois = parse_roi(roi_str)
    except Exception as e:
        print(f"Błąd w wierszu {idx}: {e}")
        continue

    # rozmiar dla każdego obrazu
    try:
        image_width = int(row['Columns'])
        image_height = int(row['Rows'])
    except:
        print(f"Brak wymiarów w wierszu {idx}, pominięto.")
        continue

    # tworzenie nazwy pliku tekstowego
    image_name = os.path.basename(row['png_path'])
    txt_name = os.path.splitext(image_name)[0] + '.txt'
    txt_path = LABELS_DIR / txt_name

    lines = []
    # obliczanie parametrów pod model YOLO
    for roi in rois:
        ymin, xmin, ymax, xmax = roi

        x_center = ((xmin + xmax) / 2) / image_width
        y_center = ((ymin + ymax) / 2) / image_height
        width = abs(xmax - xmin) / image_width
        height = abs(ymax - ymin) / image_height

        line = f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        lines.append(line)

    with open(txt_path, 'w') as f:
        f.write('\n'.join(lines))

print(f"Pliki zapisano do: {LABELS_DIR}")

Pliki zapisano do: E:\YOLO_labels\labels\train
